In [1]:
# 2023 Gabriel J. Diaz @ RIT

import os
import sys
import numpy as np
import av
import logging
import pickle
from tqdm import tqdm




import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from pathlib import Path, PurePath

logger = logging.getLogger(__name__)
logger.addHandler(logging.StreamHandler(stream=sys.stdout))

In [17]:
import sys
sys.path.append('../..')

# from retinal_flow_toolkit import .flow_source
from flow_source import *

In [18]:
os.getcwd()
sys.path

['/Users/gjdpci/Documents/GitHub/retinal_flow_toolkit/sandbox/overlay_gaze',
 '/Users/gjdpci/opt/anaconda3/envs/Pytorch_1/lib/python38.zip',
 '/Users/gjdpci/opt/anaconda3/envs/Pytorch_1/lib/python3.8',
 '/Users/gjdpci/opt/anaconda3/envs/Pytorch_1/lib/python3.8/lib-dynload',
 '',
 '/Users/gjdpci/opt/anaconda3/envs/Pytorch_1/lib/python3.8/site-packages',
 '/Users/gjdpci/opt/anaconda3/envs/Pytorch_1/lib/python3.8/site-packages/IPython/extensions',
 '/Users/gjdpci/.ipython',
 '..',
 '..',
 '..',
 '..',
 '../..',
 'core',
 '../..']

In [23]:
    
# file_path = Path('D:\\Github\\retinal_flow_toolkit\\videos\\Yoyo-LVRA.mp4')

a_file_path = Path(os.path.join('../../pupil_labs_videos/cb13'))


In [25]:
# a_file_path = os.path.join("pupil_labs_data", "GD-Short-Driving-Video")

source = pupil_labs_source(a_file_path)



In [21]:
container_in = av.open(file_path.as_posix())
average_fps = container_in.streams.video[0].average_rate
num_frames = container_in.streams.video[0].frames
time_base = container_in.streams.video[0].time_base
encoded_frame_count = container_in.streams.video[0].encoded_frame_count
container_in.sort_dts = True
container_in.flush_packets = True

container_out = av.open('gaze_overlay_on_flow.mp4', mode="w", timeout = None)


stream = container_out.add_stream("libx264", framerate = average_fps)

#stream = container_out.add_stream("h264_nvenc", framerate = average_fps) Throws pts/dts related errors.
#stream = container_out.add_stream("hevc_nvenc", framerate = average_fps) Throws pts/dts related errors.

stream.options["crf"] = "10"
stream.pix_fmt = container_in.streams.video[0].pix_fmt
stream.time_base = container_in.streams.video[0].time_base

In [ ]:
# stream.codec_context.time_base = container_in.streams.video[0].codec_context.time_base
# stream.codec_context.skip_frame = "NONKEY"

pts = []
dts = []
frame_time_base = []
relative_time = []

# for raw_frame in container_in.decode(video=0):
for raw_frame in tqdm(container_in.decode(video=0), desc="Working.", unit= 'frames',total = num_frames):
        
    pts.append(raw_frame.pts)
    dts.append(raw_frame.dts)
    frame_time_base.append(raw_frame.time_base)
    relative_time.append( np.float32(raw_frame.pts * raw_frame.time_base) )
    
    for packet in stream.encode(raw_frame):
        packet.stream = stream
        packet.time_base = time_base
        packet.pts = raw_frame.pts
        container_out.mux(packet)

for packet in stream.encode():
    packet.stream = stream
    packet.time_base = time_base
    packet.pts = raw_frame.pts
    container_out.mux(packet)

container_out.close()
container_in.close()

In [ ]:
# container_in = av.open('vid_out.mp4')
container_in = av.open('vid_out.mp4', mode="r", timeout = None)
time_base_out = container_in.streams.video[0].time_base
num_frames = container_in.streams.video[0].frames

pts_out = []
dts_out = []
frame_time_base_out = []
relative_time_out = []

count = 0
#for raw_frame in container_in.decode(video=0):
for raw_frame in tqdm(container_in.decode(video=0), desc="Working.", unit= 'frames',total = num_frames):

    pts_out.append(raw_frame.pts)
    dts_out.append(raw_frame.dts)
    frame_time_base_out.append(raw_frame.time_base)
    relative_time_out.append( np.float32(raw_frame.pts * raw_frame.time_base) )
    count = count+1
    
container_in.close()
print( count)

In [ ]:
print('Len pts:     {}'.format(len(pts)))
print('Len pts_out: {}'.format(len(pts_out)))

In [ ]:
dts = np.array(dts,dtype=np.float32)
dts_out = np.array(dts_out,dtype=np.float32)

fig, ax = plt.subplots(figsize=(15, 10), layout='constrained',dpi=300)

plt.subplot(2,2,1)
plt.plot(np.arange(0,len(pts)) ,np.array(pts) * float(time_base),'r')
plt.plot(np.arange(0,len(pts_out)) ,np.array(pts_out) * float(time_base_out),':b')
plt.ylabel('time')
plt.xlabel('pts * time_base')

plt.subplot(2,2,2)
plt.plot( relative_time,'r')
plt.plot( relative_time_out,':b')
plt.ylabel('time')
plt.xlabel('relative time')

plt.subplot(2,2,3)
plt.plot(np.arange(0,len(dts)) ,np.array(dts) * float(time_base),'r')
plt.plot(np.arange(0,len(dts_out)) ,np.array(dts_out) * float(time_base_out),':b')
plt.ylabel('time')
plt.xlabel('dts * time_base')


plt.subplot(2,2,4)
plt.plot(np.arange(0,len(frame_time_base)),frame_time_base,'r')
plt.plot(np.arange(0,len(frame_time_base_out)),frame_time_base_out,':b')
plt.ylabel('time')
plt.xlabel('frame_time_base')

plt.savefig( 'frame_timings.png')

pickle_out = 'frame_timings.pkl'
dbfile = open(pickle_out, 'wb')
data_out_dict = {"pts":pts, "pts_out":pts_out,"dts":dts,"dts_out":dts_out,"relative_time":relative_time, "relative_time_out":relative_time_out}
pickle.dump( data_out_dict, dbfile)
dbfile.close()

In [ ]:
dts = np.array(dts,dtype=np.float32)
dts_out = np.array(dts_out,dtype=np.float32)

fig, ax = plt.subplots(figsize=(15, 10), layout='constrained',dpi=300)

# plt.subplot(2,2,1)
plt.plot(np.arange(0,len(pts)) ,np.array(pts) * float(time_base),'r')

plt.plot(np.arange(0,len(pts_out)) ,np.array(pts_out) * float(time_base_out),':b')
plt.ylabel('time')
plt.xlabel('pts * time_base')


In [ ]:
np.array(pts[:5]) * float(time_base)

In [ ]:
np.array(pts_out[:5]) * float(time_base)

In [ ]:
pts[0] * float(time_base)

In [ ]:
pts_out[0] * float(time_base)

In [ ]:
container_in = av.open(file_path.as_posix())
container_out = av.open('vid_out.mp4')

In [ ]:

print('Time base A ' + str( container_in.streams.video[0].time_base))
print('Time base B ' + str(container_out.streams.video[0].time_base))

print('Num frames A ' + str(container_in.streams.video[0].frames))
print('Num frames B ' + str(container_out.streams.video[0].frames))

print('Avg rate A ' + str(container_in.streams.video[0].average_rate))
print('Avg rate B ' + str(container_out.streams.video[0].average_rate))

print('Start time A: ' + str(container_in.start_time))
print('Start time B: ' + str(container_out.start_time))

print('Duration A: ' + str(container_in.duration))
print('Duration B: ' + str(container_out.duration))


In [ ]:
container_out.streams.video[0]

In [ ]:
container_in.streams.video[0]

In [ ]:

container_out.close()
container_in.close()